# Введение

В этом задании Вы продолжите работать с данными из семинара [Articles Sharing and Reading from CI&T Deskdrop](https://www.kaggle.com/gspmoreira/articles-sharing-reading-from-cit-deskdrop).

In [1]:
import pandas as pd
import numpy as np
import math

import scipy.sparse as sp
import warnings
warnings.filterwarnings("ignore")

## Загрузка и предобработка данных

Загрузим данные и проведем предобраотку данных как на семинаре.

In [2]:
!wget -q -N https://www.dropbox.com/s/z8syrl5trawxs0n/articles.zip?dl=0 -O articles.zip
!unzip -o -q articles.zip

In [3]:
articles_df = pd.read_csv('articles/shared_articles.csv')
articles_df = articles_df[articles_df['eventType'] == 'CONTENT SHARED']
articles_df.head(2)

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en


In [4]:
interactions_df = pd.read_csv('articles/users_interactions.csv')
interactions_df.head(2)

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US


In [5]:
interactions_df.personId = interactions_df.personId.astype(str)
interactions_df.contentId = interactions_df.contentId.astype(str)
articles_df.contentId = articles_df.contentId.astype(str)

In [6]:
# зададим словарь определяющий силу взаимодействия
event_type_strength = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}

interactions_df['eventStrength'] = interactions_df.eventType.apply(lambda x: event_type_strength[x])

Оставляем только тех пользователей, которые произамодействовали более чем с пятью статьями.

In [7]:
users_interactions_count_df = (
    interactions_df
    .groupby(['personId', 'contentId'])
    .first()
    .reset_index()
    .groupby('personId').size())
print('# users:', len(users_interactions_count_df))

users_with_enough_interactions_df = \
    users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['personId']]
print('# users with at least 5 interactions:', len(users_with_enough_interactions_df))

# users: 1895
# users with at least 5 interactions: 1140


Оставляем только те взаимодействия, которые относятся к отфильтрованным пользователям.

In [8]:
interactions_from_selected_users_df = interactions_df.loc[np.in1d(interactions_df.personId,
            users_with_enough_interactions_df)]

In [9]:
print('# interactions before:', interactions_df.shape)
print('# interactions after:', interactions_from_selected_users_df.shape)

# interactions before: (72312, 9)
# interactions after: (69868, 9)


Объединяем все взаимодействия пользователя по каждой статье и сглажиываем полученный результат, взяв от него логарифм.

In [10]:
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = (
    interactions_from_selected_users_df
    .groupby(['personId', 'contentId']).eventStrength.sum()
    .apply(smooth_user_preference)
    .reset_index().set_index(['personId', 'contentId'])
)
interactions_full_df['last_timestamp'] = (
    interactions_from_selected_users_df
    .groupby(['personId', 'contentId'])['timestamp'].last()
)
        
interactions_full_df = interactions_full_df.reset_index()
interactions_full_df.head(5)

,personId,contentId,eventStrength,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.000000,1470395911
1,-1007001694607905623,-6623581327558800021,1.000000,1487240080
2,-1007001694607905623,-793729620925729327,1.000000,1472834892
3,-1007001694607905623,1469580151036142903,1.000000,1487240062
4,-1007001694607905623,7270966256391553686,1.584963,1485994324


Разобьём выборку на обучение и контроль по времени.

In [11]:
from sklearn.model_selection import train_test_split

split_ts = 1475519530
interactions_train_df = interactions_full_df.loc[interactions_full_df.last_timestamp < split_ts].copy()
interactions_test_df = interactions_full_df.loc[interactions_full_df.last_timestamp >= split_ts].copy()

print('# interactions on Train set: %d' % len(interactions_train_df))
print('# interactions on Test set: %d' % len(interactions_test_df))

interactions_train_df

# interactions on Train set: 29329
# interactions on Test set: 9777


,personId,contentId,eventStrength,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.0,1470395911
2,-1007001694607905623,-793729620925729327,1.0,1472834892
6,-1032019229384696495,-1006791494035379303,1.0,1469129122
7,-1032019229384696495,-1039912738963181810,1.0,1459376415
8,-1032019229384696495,-1081723567492738167,2.0,1464054093
...,...,...,...,...
39099,997469202936578234,9112765177685685246,2.0,1472479493
39100,998688566268269815,-1255189867397298842,1.0,1474567164
39101,998688566268269815,-401664538366009049,1.0,1474567449
39103,998688566268269815,6881796783400625893,1.0,1474567675


Для удобства подсчёта качества запишем данные в формате, где строка соответствует пользователю, а столбцы будут истинными метками и предсказаниями в виде списков.

In [12]:
interactions = (
    interactions_train_df
    .groupby('personId')['contentId'].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={'contentId': 'true_train'})
    .set_index('personId')
)

interactions['true_test'] = (
    interactions_test_df
    .groupby('personId')['contentId'].agg(lambda x: list(x))
)

# заполнение пропусков пустыми списками
interactions.loc[pd.isnull(interactions.true_test), 'true_test'] = [
    list() for x in range(len(interactions.loc[pd.isnull(interactions.true_test), 'true_test']))]

interactions.head(1)

,true_train,true_test
personId,,
-1007001694607905623,"[-5065077552540450930, -793729620925729327]","[-6623581327558800021, 1469580151036142903, 72..."


## Библиотека LightFM

Для рекомендации Вы будете пользоваться библиотекой [LightFM](https://making.lyst.com/lightfm/docs/home.html), в которой реализованы популярные алгоритмы. Для оценивания качества рекомендации, как и на семинаре, будем пользоваться метрикой *precision@10*.

In [13]:
!pip install lightfm
from lightfm import LightFM
from lightfm.evaluation import precision_at_k

     |████████████████████████████████| 317kB 9.1MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705351 sha256=818ea5f1af57560393473e7c8ef70033f2986f47d83405fc87c60f66384d6281
  Stored in directory: /root/.cache/pip/wheels/c6/64/d4/673c7277f71ac4c5ad4835b94708c01b653ef2d3aa78ef20aa
Successfully built lightfm


## Задание 1. (2 балла)

Модели в LightFM работают с разреженными матрицами. Создайте разреженные матрицы `data_train` и `data_test` (размером количество пользователей на количество статей), такие что на пересечении строки пользователя и столбца статьи стоит сила их взаимодействия, если взаимодействие было, и стоит ноль, если взаимодействия не было.

In [14]:
# Ваш код здесь

# используем класс Dataset для модели LightFM
from lightfm.data import Dataset

data = Dataset()
data.fit(interactions_full_df.personId.unique(), interactions_full_df.contentId.unique())
data_train, weights_matrix_train = data.build_interactions([tuple(i) for i in interactions_train_df.drop(['eventStrength', 'last_timestamp'], axis = 1).values])
data_test, weights_matrix_test = data.build_interactions([tuple(i) for i in interactions_test_df.drop(['eventStrength', 'last_timestamp'], axis = 1).values])

## Задание 2. (1 балл)

Обучите модель LightFM с `loss='warp'` и посчитайте *precision@10* на тесте.

In [51]:
# Ваш код здесь

# сразу зафиксируем random_state для получения одинаковых результатов при повторном обучении модели
model = LightFM(loss='warp', random_state=7)
model.fit(data_train, sample_weight = weights_matrix_train)
precision = precision_at_k(model, data_test, data_train, k=10).mean()

print('Precision@10:', precision)

Precision@10: 0.004480651


## Задание 3. (3 балла)

При вызове метода `fit` LightFM позволяет передавать в `item_features` признаковое описание объектов. Воспользуемся этим. Будем получать признаковое описание из текста статьи в виде [TF-IDF](https://ru.wikipedia.org/wiki/TF-IDF) (можно воспользоваться `TfidfVectorizer` из scikit-learn). Создайте матрицу `feat` размером количество статей на размер признакового описание и обучите LightFM с `loss='warp'` и посчитайте precision@10 на тесте.

In [16]:
# Ваш код здесь

# получим список всех текстов статей (всего 3047)
corpus = list(articles_df.text)
len(corpus)

3047

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=300)
feat = vectorizer.fit_transform(corpus)

In [18]:
feat.shape

(3047, 300)

In [52]:
model = LightFM(loss='warp', random_state=7)
model.fit(data_train, sample_weight = weights_matrix_train, item_features=feat)
precision = precision_at_k(model, data_test, data_train, item_features=feat, k=10).mean()

print('Precision@10:', precision)

Precision@10: 0.010794298


__Подбор гиперпараметра max_features:__ \\

max_features = 1000 - precision ~ 0.002 \\
max_features = 500 --- precision ~ 0.003 \\
max_features = 400 --- precision ~ 0.001 \\
max_features = 300 --- precision ~ 0.01 \\
max_features = 200 --- precision ~ 0.002 \\
Поэтому оставляем max_features = 300



## Задание 4. (2 балла)

В задании 3 мы использовали сырой текст статей. В этом задании необходимо сначала сделать предобработку текста (привести к нижнему регистру, убрать стоп слова, привести слова к нормальной форме и т.д.), после чего обучите модель и оценить качество на тестовых данных.

In [21]:
# Ваш код здесь

# найдем языки, встречающиеся в текстах статей, чтобы добавить стоп-слова из разных языков
!pip install langdetect

     |████████████████████████████████| 983kB 8.7MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.9-cp37-none-any.whl size=993223 sha256=0578ff05848dcd721eed5fe698e6106feb257b1f2df539fcfaaf51ac2e23ab6a
  Stored in directory: /root/.cache/pip/wheels/7e/18/13/038c34057808931c7ddc6c92d3aa015cf1a498df5a70268996
Successfully built langdetect


In [22]:
from langdetect import detect

lang = set([detect(text) for text in corpus])
lang

{'ca', 'en', 'es', 'pt'}

In [23]:
# тексты статей: catalan, english, spanish, portugues

In [23]:
# предобработка текста

import string
import torchtext

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.tokenize import word_tokenize
# добавим лемматизацию слов
from nltk.stem import WordNetLemmatizer 
# уберем частые слова
from nltk.corpus import stopwords

def digit(s): # функция проверки на наличие цифр в строке 
    return all([c.isdigit() for c in s])

# в пакете stopwords нет языка 'catalan', поэтому слова этого языка не добавляем в список стоп-слов
stop_words = set(stopwords.words('english')) \
                .union(set(stopwords.words('spanish'))) \
                .union(set(stopwords.words('portuguese')))

def process_text(text):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in word_tokenize(text.lower()) 
            if word not in stop_words 
            and (lemmatizer.lemmatize(word) not in string.punctuation)
            and not digit(word)]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
# из каждой статьи уберем частые слова, пунктуацию и числа
process_corpus = [' '.join(process_text(text)) for text in corpus]

In [37]:
vectorizer = TfidfVectorizer(max_features = 300, ngram_range=(1, 2), min_df=0.08, sublinear_tf=True) 
feat = vectorizer.fit_transform(process_corpus)

model = LightFM(loss='warp', random_state=7)
model.fit(data_train, sample_weight = weights_matrix_train, item_features=feat)
precision = precision_at_k(model, data_test, data_train, item_features=feat, k=10).mean()

print('Precision@10:', precision)

Precision@10: 0.010794298


__Подбор гиперпараметров TfidfVectorizer:__

TfidfVectorizer(max_features = 300, ngram_range=(1, 2), min_df=0.08) (max_df $\in$ {0.9, 0.8, 0.7, 0.6}) --- precision = 0.0060081473  \\
TfidfVectorizer(max_features = 300, ngram_range=(1, 2), min_df=0.08, sublinear_tf=True) --- precision = 0.010794298 \\
TfidfVectorizer(max_features = 300, ngram_range=(1, 2), max_df=0.5, min_df=0.08) --- precision = 0.0077393074 \\
TfidfVectorizer(max_features = 300, ngram_range=(1, 2), max_df=0.9, min_df=0.1) --- precision = 0.0088594705

Улучшилось ли качество предсказания? \\

Качество почти не изменилось, но немного выросло (0.010794298 против 0.010081466)

## Задание 5. (2 балла)

Подберите гиперпараметры модели LightFM (`n_components` и др.) для улучшения качества модели.

In [59]:
# Ваш код здесь
for i in range(1, 21):
  model = LightFM(loss='warp', no_components=i, random_state=7)
  model.fit(data_train, sample_weight = weights_matrix_train, item_features=feat)
  precision = precision_at_k(model, data_test, data_train, item_features=feat, k=10).mean()

  print('Precision@10:', precision, 'no_components =', i)

Precision@10: 0.003869654 no_components = 1
Precision@10: 0.0066191447 no_components = 2
Precision@10: 0.0027494908 no_components = 3
Precision@10: 0.009368637 no_components = 4
Precision@10: 0.008044806 no_components = 5
Precision@10: 0.007739308 no_components = 6
Precision@10: 0.0061099795 no_components = 7
Precision@10: 0.0054989816 no_components = 8
Precision@10: 0.009368637 no_components = 9
Precision@10: 0.010794298 no_components = 10
Precision@10: 0.0035641547 no_components = 11
Precision@10: 0.009164969 no_components = 12
Precision@10: 0.00407332 no_components = 13
Precision@10: 0.007942974 no_components = 14
Precision@10: 0.009063137 no_components = 15
Precision@10: 0.007535642 no_components = 16
Precision@10: 0.005193483 no_components = 17
Precision@10: 0.009063136 no_components = 18
Precision@10: 0.008452139 no_components = 19
Precision@10: 0.0018329938 no_components = 20


In [62]:
model = LightFM(loss='warp', no_components=10, learning_schedule='adadelta', random_state=7)
model.fit(data_train, sample_weight = weights_matrix_train, item_features=feat)
precision = precision_at_k(model, data_test, data_train, item_features=feat, k=10).mean()

print('Precision@10:', precision)

Precision@10: 0.008452138


In [65]:
for i in range(1, 21):
  model = LightFM(loss='warp', no_components=10, item_alpha=0.01*i, random_state=7)
  model.fit(data_train, sample_weight = weights_matrix_train, item_features=feat)
  precision = precision_at_k(model, data_test, data_train, item_features=feat, k=10).mean()

  print('Precision@10:', precision, 'item_alpha =', 0.01*i)

Precision@10: 0.007535642 item_alpha = 0.01
Precision@10: 0.004378819 item_alpha = 0.02
Precision@10: 0.005702648 item_alpha = 0.03
Precision@10: 0.0032586558 item_alpha = 0.04
Precision@10: 0.0029531568 item_alpha = 0.05
Precision@10: 0.0023421592 item_alpha = 0.06
Precision@10: 0.0038696537 item_alpha = 0.07
Precision@10: 0.0048879837 item_alpha = 0.08
Precision@10: 0.0041751526 item_alpha = 0.09
Precision@10: 0.004378819 item_alpha = 0.1
Precision@10: 0.003360489 item_alpha = 0.11
Precision@10: 0.004480652 item_alpha = 0.12
Precision@10: 0.004480652 item_alpha = 0.13
Precision@10: 0.005397149 item_alpha = 0.14
Precision@10: 0.0031568226 item_alpha = 0.15
Precision@10: 0.003665988 item_alpha = 0.16
Precision@10: 0.0045824847 item_alpha = 0.17
Precision@10: 0.00407332 item_alpha = 0.18
Precision@10: 0.0030549897 item_alpha = 0.19
Precision@10: 0.0035641547 item_alpha = 0.2


In [66]:
for i in range(1, 21):
  model = LightFM(loss='warp', no_components=10, user_alpha=0.01*i, random_state=7)
  model.fit(data_train, sample_weight = weights_matrix_train, item_features=feat)
  precision = precision_at_k(model, data_test, data_train, item_features=feat, k=10).mean()

  print('Precision@10:', precision, 'user_alpha =', 0.01*i)

Precision@10: 0.010488799 user_alpha = 0.01
Precision@10: 0.0105906315 user_alpha = 0.02
Precision@10: 0.010488798 user_alpha = 0.03
Precision@10: 0.010386966 user_alpha = 0.04
Precision@10: 0.010794298 user_alpha = 0.05
Precision@10: 0.011099796 user_alpha = 0.06
Precision@10: 0.010285133 user_alpha = 0.07
Precision@10: 0.010997963 user_alpha = 0.08
Precision@10: 0.010794298 user_alpha = 0.09
Precision@10: 0.010997963 user_alpha = 0.1
Precision@10: 0.011303463 user_alpha = 0.11
Precision@10: 0.011099796 user_alpha = 0.12
Precision@10: 0.011099796 user_alpha = 0.13
Precision@10: 0.011099796 user_alpha = 0.14
Precision@10: 0.010896131 user_alpha = 0.15
Precision@10: 0.010794298 user_alpha = 0.16
Precision@10: 0.010183299 user_alpha = 0.17
Precision@10: 0.010692464 user_alpha = 0.18
Precision@10: 0.010692464 user_alpha = 0.19
Precision@10: 0.010285133 user_alpha = 0.2


In [67]:
for i in range(1, 21):
  model = LightFM(loss='warp', no_components=10, user_alpha=0.11, max_sampled=i, random_state=7)
  model.fit(data_train, sample_weight = weights_matrix_train, item_features=feat)
  precision = precision_at_k(model, data_test, data_train, item_features=feat, k=10).mean()

  print('Precision@10:', precision, 'max_sampled =', i)

Precision@10: 0.010896131 max_sampled = 1
Precision@10: 0.011201629 max_sampled = 2
Precision@10: 0.010183299 max_sampled = 3
Precision@10: 0.010692465 max_sampled = 4
Precision@10: 0.010896131 max_sampled = 5
Precision@10: 0.011303463 max_sampled = 6
Precision@10: 0.010997963 max_sampled = 7
Precision@10: 0.011201629 max_sampled = 8
Precision@10: 0.011303463 max_sampled = 9
Precision@10: 0.011303463 max_sampled = 10
Precision@10: 0.010285133 max_sampled = 11
Precision@10: 0.011303463 max_sampled = 12
Precision@10: 0.011405295 max_sampled = 13
Precision@10: 0.011201629 max_sampled = 14
Precision@10: 0.010692464 max_sampled = 15
Precision@10: 0.011812627 max_sampled = 16
Precision@10: 0.010896131 max_sampled = 17
Precision@10: 0.010997963 max_sampled = 18
Precision@10: 0.010081467 max_sampled = 19
Precision@10: 0.011099796 max_sampled = 20


In [68]:
# итоговое качество 
model = LightFM(loss='warp', no_components=10, user_alpha=0.11, max_sampled=16, random_state=7)
model.fit(data_train, sample_weight = weights_matrix_train, item_features=feat)
precision = precision_at_k(model, data_test, data_train, item_features=feat, k=10).mean()

print('Precision@10:', precision)

Precision@10: 0.011812627


__Подбор гиперпараметров LightFM:__

* no_components: \\
Максимальное значение precision = 0.010794298 при значении параметра 10  \\

* learning_schedule: \\
precision = 0.008248473 при learning_schedule='adadelta' - качество понизилось, рассматриваем только learning_schedule='adagrad' \\

* item_alpha: \\
precision порядка 0.001 при изменении параметра от 0.01 до 0.2 \\

* user_alpha: \\
max_precision = 0.011303463 при user_alpha = 0.11

* max_sampled: \\
max_precision = 0.011812627 при max_sampled = 16 \\

Итоговое качество: precision@10 = 0.011812627

## Бонусное задание. (3 балла)

Выше мы использовали достаточно простое представление текста статьи в виде TF-IDF. В этом задании Вам нужно представить текст статьи (можно вместе с заголовком) в виде эмбеддинга полученного с помощью рекуррентной сети или трансформера (можно использовать любую предобученную модель, которая Вам нравится). Обучите модель с ипользованием этих эмеддингов и сравните результаты с предыдущими.

In [29]:
# Ваш код здесь